In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, Imputer
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.ml.stat import Correlation
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Buat sesi Spark
spark = SparkSession.builder.appName("DecisionTreeRegressor").config("spark.executor.heartbeatInterval", "100s").config("spark.network.timeout", "1200s").getOrCreate()

In [3]:
# Ubah 'file_path' dengan path dataset Anda
file_path = "kapita.csv"
dataset = spark.read.csv(file_path, header=True, inferSchema=True)

In [4]:
# Tampilkan data dari tabel
dataset.show()

+-----------------------+---------+---------+
|Provinsi/Kabupaten/Kota|Laki-laki|Perempuan|
+-----------------------+---------+---------+
|                   ACEH|    14025|     8495|
|               SIMEULUE|    11709|     3560|
|               Simeulue|    11709|     3560|
|           ACEH SINGKIL|    15346|     6062|
|           Aceh Singkil|    15346|     6062|
|           ACEH SELATAN|    11317|     7294|
|           Aceh Selatan|    11317|     7294|
|          ACEH TENGGARA|    12210|     8005|
|          Aceh Tenggara|    12210|     8005|
|             ACEH TIMUR|    14215|     5937|
|             Aceh Timur|    14215|     5937|
|            ACEH TENGAH|    11755|     9735|
|            Aceh Tengah|    11755|     9735|
|             ACEH BARAT|    14524|     5852|
|             Aceh Barat|    14524|     5852|
|             ACEH BESAR|    12821|     9205|
|             Aceh Besar|    12821|     9205|
|                  PIDIE|    13249|     9134|
|                  Pidie|    13249

In [5]:
# Konversi kolom-kolom "Laki-laki" dan "Perempuan" ke tipe data numerik
dataset = dataset.withColumn("Laki-laki", dataset["Laki-laki"].cast("double"))
dataset = dataset.withColumn("Perempuan", dataset["Perempuan"].cast("double"))

In [6]:
# Ganti nilai null dengan nilai rata-rata menggunakan Imputer
imputer = Imputer(inputCols=["Laki-laki", "Perempuan"], outputCols=["Laki-laki_imputed", "Perempuan_imputed"])
dataset = imputer.fit(dataset).transform(dataset)

In [7]:
# Agregasi data untuk mendapatkan total pengeluaran per kapita
dataset = dataset.withColumn("Total_Pengeluaran", col("Laki-laki_imputed") + col("Perempuan_imputed"))
dataset = dataset.withColumn("Pengeluaran_Per_Kapita", col("Total_Pengeluaran") / (col("Laki-laki_imputed") + col("Perempuan_imputed")))

NameError: name 'pipeline_male' is not defined

In [10]:
# Tentukan kolom fitur yang akan digunakan
feature_columns = ["Laki-laki_imputed", "Perempuan_imputed"]

In [11]:
# Inisialisasi VectorAssembler untuk menggabungkan fitur-fitur menjadi vektor fitur tunggal
feature_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [12]:
# Inisialisasi DecisionTreeRegressor
dt_regressor = DecisionTreeRegressor(labelCol="Pengeluaran_Per_Kapita", featuresCol="features")

In [13]:
# Bangun pipeline dengan langkah-langkah yang telah ditentukan
pipeline = Pipeline(stages=[imputer, feature_assembler, dt_regressor])

In [14]:
# Bagi data menjadi set pelatihan dan set pengujian
(training_data, testing_data) = dataset.randomSplit([0.8, 0.2], seed=42)

In [15]:
# Latih model dengan set pelatihan
model = pipeline.fit(training_data)

In [16]:
# Buat prediksi dengan set pengujian
predictions = model.transform(testing_data)

In [17]:
# Evaluasi performa model
evaluator = RegressionEvaluator(labelCol="Pengeluaran_Per_Kapita", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 0.0


In [18]:
# Tampilkan hasil prediksi
result = predictions.select("Provinsi/Kabupaten/Kota", "Pengeluaran_Per_Kapita", "prediction")
result.show()

+-----------------------+----------------------+----------+
|Provinsi/Kabupaten/Kota|Pengeluaran_Per_Kapita|prediction|
+-----------------------+----------------------+----------+
|        ACEH BARAT DAYA|                   1.0|       1.0|
|           ACEH SINGKIL|                   1.0|       1.0|
|            ACEH TENGAH|                   1.0|       1.0|
|                   ALOR|                   1.0|       1.0|
|              Aceh Jaya|                   1.0|       1.0|
|            Aceh Tengah|                   1.0|       1.0|
|                 Asahan|                   1.0|       1.0|
|          BANDUNG BARAT|                   1.0|       1.0|
|                 BANJAR|                   1.0|       1.0|
|           BANJARNEGARA|                   1.0|       1.0|
|               BANTAENG|                   1.0|       1.0|
|                 BANTUL|                   1.0|       1.0|
|               BANYUMAS|                   1.0|       1.0|
|           BARITO TIMUR|               

In [19]:
# Menampilkan schema dataset
dataset.printSchema()

root
 |-- Provinsi/Kabupaten/Kota: string (nullable = true)
 |-- Laki-laki: double (nullable = true)
 |-- Perempuan: double (nullable = true)
 |-- Laki-laki_imputed: double (nullable = true)
 |-- Perempuan_imputed: double (nullable = true)
 |-- Total_Pengeluaran: double (nullable = true)
 |-- Pengeluaran_Per_Kapita: double (nullable = true)



In [20]:
# Menampilkan statistik deskriptif
dataset.describe().show()

+-------+-----------------------+------------------+-----------------+------------------+------------------+------------------+----------------------+
|summary|Provinsi/Kabupaten/Kota|         Laki-laki|        Perempuan| Laki-laki_imputed| Perempuan_imputed| Total_Pengeluaran|Pengeluaran_Per_Kapita|
+-------+-----------------------+------------------+-----------------+------------------+------------------+------------------+----------------------+
|  count|                   1072|              1067|             1067|              1072|              1072|              1072|                  1072|
|   mean|                   NULL|14805.646672914714|8643.551077788192| 14805.64667291471| 8643.551077788188|23449.197750702904|                   1.0|
| stddev|                   NULL| 3596.804225135792|3193.448631095079|3588.3985015042836|3185.9855486073525|  6128.67597157729|                   0.0|
|    min|                   ACEH|            4146.0|           1582.0|            4146.0|     

In [21]:
# Menampilkan jumlah baris dalam dataset
print("Jumlah Baris: ", dataset.count())

Jumlah Baris:  1072


In [22]:
# Menampilkan statistik pengeluaran per kapita
dataset.select("Pengeluaran_Per_Kapita").summary().show()

+-------+----------------------+
|summary|Pengeluaran_Per_Kapita|
+-------+----------------------+
|  count|                  1072|
|   mean|                   1.0|
| stddev|                   0.0|
|    min|                   1.0|
|    25%|                   1.0|
|    50%|                   1.0|
|    75%|                   1.0|
|    max|                   1.0|
+-------+----------------------+



In [23]:
# Menampilkan statistik pengeluaran laki-laki dan perempuan
dataset.select("Laki-laki", "Perempuan").summary().show()

+-------+------------------+-----------------+
|summary|         Laki-laki|        Perempuan|
+-------+------------------+-----------------+
|  count|              1067|             1067|
|   mean|14805.646672914714|8643.551077788192|
| stddev| 3596.804225135792|3193.448631095079|
|    min|            4146.0|           1582.0|
|    25%|           12455.0|           6429.0|
|    50%|           14707.0|           8435.0|
|    75%|           17085.0|          10304.0|
|    max|           28077.0|          23626.0|
+-------+------------------+-----------------+



In [42]:
# Menggabungkan fitur-fitur menjadi vektor fitur tunggal
vector_assembler = VectorAssembler(inputCols=["Laki-laki", "Perempuan"], outputCol="features")
dataset_features = vector_assembler.transform(dataset)

In [43]:
# Membersihkan nilai null dari dataset
cleaned_dataset = dataset.na.drop()

In [44]:
# Menggunakan VectorAssembler pada dataset yang telah dibersihkan
dataset_features = vector_assembler.transform(cleaned_dataset)

In [44]:
# Menghitung korelasi antara fitur
pearson_corr = Correlation.corr(dataset_features, "features", method="pearson").head()
print("Korelasi Pearson: ", str(pearson_corr[0]))

NameError: name 'dataset_features' is not defined

In [43]:
# Visualisasi matriks korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(pearson_corr[0].toArray(), annot=True, fmt=".2f", cmap="coolwarm", xticklabels=dataset.columns[1:], yticklabels=dataset.columns[1:])
plt.title("Pearson Correlation Matrix")
plt.show()

NameError: name 'pearson_corr' is not defined

<Figure size 1000x800 with 0 Axes>